# day 16

https://adventofcode.com/2020/day/16

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day16.txt')

LOGGER = logging.getLogger('day16')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
import re

import numpy as np

def parse_data(data):
    field_info, my_ticket, nearby_tickets = data.split('\n\n')
    field_info = [re.match('([\w ]+): (\d+)\-(\d+) or (\d+)\-(\d+)', line.strip()).groups()
                  for line in field_info.split('\n')
                  if line]
    field_info = {field: [[int(x0), int(x1)], [int(y0), int(y1)]]
                  for (field, x0, x1, y0, y1) in field_info}
    my_ticket = [int(_) for _ in my_ticket.split('\n')[1].split(',')]
    nearby_tickets = [[int(_) for _ in ticket.split(',')]
                      for ticket in nearby_tickets.split('\n')[1:]]
    nearby_tickets = np.array(nearby_tickets)
    return (field_info,
            my_ticket,
            nearby_tickets)

In [ ]:
parse_data(test_data)

#### function def

In [ ]:
import collections
fi, mt, nt = parse_data(test_data)
c = collections.Counter(nt.flatten().tolist())
c

In [ ]:
def q_1(data):
    fi, mt, nt = parse_data(data)
    ticket_vals = collections.Counter(nt.flatten().tolist())
    allowed_vals = set()
    for (f, ((x0, x1), (y0, y1))) in fi.items():
        for i in range(x0, x1 + 1):
            allowed_vals.add(i)
        for j in range(y0, y1 + 1):
            allowed_vals.add(j)
    return sum(k * v for (k, v) in ticket_vals.items()
               if k not in allowed_vals)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 71
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

In [ ]:
test_data = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9"""

#### function def

In [ ]:
import pandas as pd

def q_2(data, is_test=False):
    fi, mt, nt = parse_data(data)
    ticket_vals = collections.Counter(nt.flatten().tolist())
    allowed_vals = set()
    for (f, ((x0, x1), (y0, y1))) in fi.items():
        for i in range(x0, x1 + 1):
            allowed_vals.add(i)
        for j in range(y0, y1 + 1):
            allowed_vals.add(j)
    
    unallowed_vals = {k for (k, v) in ticket_vals.items() if k not in allowed_vals}
    
    # drop bad ones
    nt = pd.DataFrame(nt)
    nt = nt[~nt.isin(unallowed_vals).any(axis=1)].copy()
    
    existing_vals = {colname: set(col.unique()) for (colname, col) in nt.items()}
    allowed_vals = {f: set() for f in fi}
    for (f, ((x0, x1), (y0, y1))) in fi.items():
        for i in range(x0, x1 + 1):
            allowed_vals[f].add(i)
        for j in range(y0, y1 + 1):
            allowed_vals[f].add(j)
    
    field_to_col_idx_map = {}
    unassigned_field = set(fi.keys())
    
    while unassigned_field:
        d_f_to_idx = {f: [] for f in allowed_vals if f not in field_to_col_idx_map}
        d_idx_to_f = {ci: [] for ci in existing_vals if ci not in field_to_col_idx_map.values()}
        
        for (field, av) in allowed_vals.items():
            if field in field_to_col_idx_map:
                continue
            for (col_idx, col_vals) in existing_vals.items():
                if col_idx in field_to_col_idx_map.values():
                    continue
                if col_vals.issubset(av):
                    d_f_to_idx[field].append(col_idx)
                    d_idx_to_f[col_idx].append(field)
        
        for (field, col_idx_set) in d_f_to_idx.items():
            if len(col_idx_set) == 1:
                field_to_col_idx_map[field] = col_idx_set[0]
                unassigned_field.discard(field)
                
        for (col_idx, field_set) in d_idx_to_f.items():
            if len(field_set) == 1:
                field = field_set[0]
                field_to_col_idx_map[field] = col_idx
                unassigned_field.discard(field)
        LOGGER.debug(f"d_f_to_idx = {d_f_to_idx}")
        LOGGER.debug(f"d_idx_to_f = {d_idx_to_f}")
        LOGGER.debug(f"field_to_col_idx_map = {field_to_col_idx_map}")
    
    if is_test:
        _class = mt[field_to_col_idx_map['class']]
        row = mt[field_to_col_idx_map['row']]
        seat = mt[field_to_col_idx_map['seat']]
        return _class * row * seat
    
    x = 1
    for field, idx in field_to_col_idx_map.items():
        if field.startswith('departure'):
            x *= mt[idx]
    return x

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data, is_test=True) == 12 * 11 * 13
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin